# Dogbone

In [1]:
import numpy as np
import gmsh
import sys

# Initialize the Gmsh API
gmsh.initialize()
gmsh.model.list()
gmsh.model.add("dogbone")

# meshsize
lc = 2.0

# coordinates, meshsize, tag
gmsh.model.geo.add_point(0 ,0,0,lc,1)
gmsh.model.geo.add_point(60,0,0,lc,2)
gmsh.model.geo.add_point(60,20,0,lc,3)
gmsh.model.geo.add_point(60,30,0,lc,4)
gmsh.model.geo.add_point(50,30,0,lc,5)
gmsh.model.geo.add_point(50,110,0,lc,6)
gmsh.model.geo.add_point(60,110,0,lc,7)
gmsh.model.geo.add_point(60,120,0,lc,8)
gmsh.model.geo.add_point(60,140,0,lc,9)
gmsh.model.geo.add_point(0,140,0,lc,10)
gmsh.model.geo.add_point(0,120,0,lc,11)
gmsh.model.geo.add_point(0,110,0,lc,12)
gmsh.model.geo.add_point(10,110,0,lc,13)
gmsh.model.geo.add_point(10,30,0,lc,14)
gmsh.model.geo.add_point(0,30,0,lc,15)
gmsh.model.geo.add_point(0,20,0,lc,16)

gmsh.model.geo.add_line(1,2,1)
gmsh.model.geo.add_line(2,3,2)
gmsh.model.geo.add_circle_arc(3,4,5,3)
gmsh.model.geo.add_line(5,6,4)
gmsh.model.geo.add_circle_arc(6,7,8,5)
gmsh.model.geo.add_line(8,9,6)
gmsh.model.geo.add_line(9,10,7)
gmsh.model.geo.add_line(10,11,8)
gmsh.model.geo.add_circle_arc(11,12,13,9)
gmsh.model.geo.add_line(13,14,10)
gmsh.model.geo.add_circle_arc(14,15,16,11)
gmsh.model.geo.add_line(16,1,12)

# list of points, tag
gmsh.model.geo.add_curve_loop([1,2,3,4,5,6,7,8,9,10,11,12],1)

# list of curvee loops, tag
gmsh.model.geo.add_plane_surface([1],1)

# dimension, list, tag
gmsh.model.add_physical_group(1, [1], name="constrained edge")
gmsh.model.add_physical_group(1, [7], name="load edge")
gmsh.model.add_physical_group(2, [1], name="dogbone surface")


gmsh.model.setColor([(1,1)],0,255,255,255,True)
gmsh.model.setColor([(1,7)],255,0,255,255,True)
gmsh.model.setColor([(2,1)],255,255,0,255,True)

# Synchronize the model to build the geometry
gmsh.model.geo.synchronize()

In [3]:
# generate a 2d mesh
gmsh.model.mesh.generate(2)

# partitioned the mesh
gmsh.model.mesh.partition(4)

In [5]:

# Synchronize the model to build the geometry
gmsh.model.geo.synchronize()

# visualize the model, whatever you do here it is avaialble later in the script
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

In [30]:
# get the list of elements
elementTypes, elementTags, nodeTags = gmsh.model.mesh.get_elements(-1,-1)  

# show the properties for each time
for elementType in elementTypes:
    print(gmsh.model.mesh.get_element_properties(elementType))

('Line 2', 1, 1, 2, array([-1.,  1.]), 2)
('Triangle 3', 2, 1, 3, array([0., 0., 1., 0., 0., 1.]), 3)
('Point', 0, 0, 1, array([0.]), 1)
